In [1]:
import pandas as pd
import numpy as np
import os
import shutil
from os import path
import joblib
from tqdm.notebook import tqdm
from nltk import ngrams
from collections import Counter
from glob import glob

# GET SUBGENRES FOLDERS

In [11]:
df_corpus_genred = pd.read_csv(r'/home/crazyjeannot/Documents/master/M2/master_thesis/meta_data/corpus.csv')
df_corpus_genred.set_index(['doc_name'], inplace=True)

In [12]:
df_corpus_genred.genre.value_counts()#[:14]

genre
roman d'aventures               364
policier                        307
roman historique                207
nouvelles                       186
cycles et séries                174
littérature jeunesse            146
mémoires et autobiographie      136
roman sentimental               133
érotisme                        107
fantastique                      66
récit de voyage                  34
épistolaire                      33
science-fiction                  31
non-fiction                      25
fragments                        16
journal intime                    8
biographie                        5
roman historique                  3
 mémoires et autobiographie       2
 nouvelles                        2
 roman sentimental                1
 littérature jeunesse             1
ciné-roman                        1
littérature jeunesse              1
littérature à contraintes         1
Name: count, dtype: int64

In [13]:
genres_ok = ["roman d'aventures",
'roman historique',
'policier',
'littérature jeunesse',
'mémoires et autobiographie',
'roman sentimental', 
'érotisme', 
'fantastique', 
'récit de voyage',
'épistolaire', 
'science-fiction', 
'non-fiction']

In [14]:
df_genred_ok = df_corpus_genred[df_corpus_genred.genre.isin(genres_ok)]

In [15]:
path_folder = "/home/crazyjeannot/Documents/doctorat/dev/AWARDS/corpus/"
subgenres = list(df_genred_ok.genre)
filenames = list(df_genred_ok.index)

In [18]:
def organize_by_subgenre(file_names, subgenres, path_folder):
    # Create subgenre folders
    for subgenre in subgenres:
        subgenre_folder = path.join(path_folder, subgenre)
        os.makedirs(subgenre_folder, exist_ok=True)

    # Iterate through files in the main folder
    for file_name in os.listdir(path_folder):
        file_path = path.join(path_folder, file_name)
        file_name = path.splitext(path.basename(file_name))[0]
        # Check if the item is a file
        if path.isfile(file_path):
            # Check if the file_name is in the provided list
            if file_name in file_names:
                # Get the index of the file_name in the list
                index = file_names.index(file_name)

                # Get the corresponding subgenre
                file_subgenre = subgenres[index]

                # Create destination folder for the subgenre if not exists
                subgenre_folder = path.join(path_folder, file_subgenre)
                dest_folder = path.join(subgenre_folder, file_name)

                # Copy the file to the subgenre folder
                shutil.copy2(file_path, dest_folder)
                print(f"File {file_name} copied to {file_subgenre} folder.")


In [19]:
organize_by_subgenre(filenames, subgenres, path_folder)

File 2008_Nunez-Laurent_Les-recidivistes copied to mémoires et autobiographie folder.
File 1960_San-Antonio_San-Antonio-renvoie-la-balle copied to érotisme folder.
File 2011_Vargas-Fred_L-Armee-furieuse copied to policier folder.
File 2002_Gaude-Laurent_La-Mort-du-roi-Tsongor copied to roman historique folder.
File 1922_Feval-Paul-fils_Le-Bossu_livre-deuxième-4-Mariquita copied to littérature jeunesse folder.
File 1974_A.D.G._Je-suis-un-roman-noir copied to policier folder.
File 2008_Grange-Jean-Christophe_Miserere copied to policier folder.
File 1934_Le-Faure-Georges_Le-brigadier-Floridor copied to roman d'aventures folder.
File 1986_Daeninckx-Didier_Le-bourreau-et-son-double copied to policier folder.
File 1926_Crevel-René_La-mort-difficile copied to mémoires et autobiographie folder.
File 1978_Delly_Le-testament-de-Mr.-d'Erquoy copied to roman sentimental folder.
File 1907_Zevaco-Michel_Les-Pardaillan copied to roman d'aventures folder.
File 1959_Magali_L'absente copied to roman sen

# GET MOTIFS SUBGENRES

In [2]:
MOTIFS_sentences = joblib.load('/home/crazyjeannot/Documents/doctorat/dev/AWARDS/data/MOTIFS_AWARDS.pkl')
chapitres_index_sentences = joblib.load('/home/crazyjeannot/Documents/doctorat/data/data_chapitres/lemmatizers/fall_pass/main_list_index_stanza_sentences_chapitres.pkl')

In [20]:
path_aventures = "/home/crazyjeannot/Documents/doctorat/dev/AWARDS/corpus/aventures/*"
path_historique = "/home/crazyjeannot/Documents/doctorat/dev/AWARDS/corpus/historique/*"
path_policier = "/home/crazyjeannot/Documents/doctorat/dev/AWARDS/corpus/policier/*"
path_fantastique = "/home/crazyjeannot/Documents/doctorat/dev/AWARDS/corpus/fantastique/*"
path_erotisme = "/home/crazyjeannot/Documents/doctorat/dev/AWARDS/corpus/érotisme/*"
path_jeunesse = "/home/crazyjeannot/Documents/doctorat/dev/AWARDS/corpus/jeunesse/*"
path_autobio = "/home/crazyjeannot/Documents/doctorat/dev/AWARDS/corpus/autobiographie/*"
path_nonfiction = "/home/crazyjeannot/Documents/doctorat/dev/AWARDS/corpus/non-fiction/*"
path_sentimental = "/home/crazyjeannot/Documents/doctorat/dev/AWARDS/corpus/sentimental/*"
path_sf = "/home/crazyjeannot/Documents/doctorat/dev/AWARDS/corpus/sf/*"
path_epistolaire = "/home/crazyjeannot/Documents/doctorat/dev/AWARDS/corpus/épistolaire/*"
path_voyage = "/home/crazyjeannot/Documents/doctorat/dev/AWARDS/corpus/voyage/*"

In [4]:
def generate_n_grams_dico(n):
    dico_list_ngrams = {}
    for i in range(1, n+1):
        list_name = f"list_{i}_gram"
        dico_list_ngrams[list_name]=[]

    return dico_list_ngrams

In [57]:
def ngram_list(sentences, n_gram_len, m_most_common):
    # Flatten the list of sentences and split into words
    words = [word for roman in sentences for sentence in roman for word in sentence.split()]
    # Generate n-grams
    n_grams_nltk = ngrams(words, n_gram_len)
    # get n-grams str
    ngram_str = ['_'.join(ngram) for ngram in n_grams_nltk]
    # Count n-grams
    ngram_counts = Counter(ngram_str)
    return list(dict(ngram_counts.most_common(m_most_common)).keys())

In [59]:
def get_ngrams(roman, len_ngrams, m_most_common):
    DICT_TEMP = generate_n_grams_dico(len_ngrams)#key = "list_{i}_gram" avec list vide   
    
    for i in tqdm(range(1, len_ngrams+1)):
            DICT_TEMP[f"list_{i}_gram"].extend(ngram_list(roman, i, m_most_common))# too big for memory ?
            
    return DICT_TEMP

In [65]:
def get_motifs_by_subgenres(path_name, list_index, list_motifs, len_ngrams=5, m_most_common=10):    
    df_main = pd.DataFrame()
    list_subgenre = []

    for doc in tqdm(glob(path_name)):
        doc_name = path.basename(doc)
        if doc_name in list_index:
            index_of_element = list_index.index(doc_name)
            list_subgenre.append(list_motifs[index_of_element])
    
    DICT_NGRAMS = get_ngrams(list_subgenre, len_ngrams, m_most_common)
    df = pd.DataFrame.from_dict(DICT_NGRAMS, orient='index')#.transpose()
    return df

In [66]:
df_aventures = get_motifs_by_subgenres(path_aventures, chapitres_index_sentences, MOTIFS_sentences)

  0%|          | 0/84 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [67]:
df_aventures

,0,1,2,3,4,5,6,7,8,9
list_1_gram,NOUN,VERB,le,de,il,ADJ,un,son,et,être
list_2_gram,le_NOUN,NOUN_de,un_NOUN,NOUN_ADJ,de_NOUN,son_NOUN,NOUN_VERB,il_VERB,à_le,ADJ_NOUN
list_3_gram,le_NOUN_de,de_le_NOUN,NOUN_de_NOUN,VERB_le_NOUN,NOUN_de_le,à_le_NOUN,NOUN_le_NOUN,le_NOUN_ADJ,le_NOUN_VERB,VERB_à_le
list_4_gram,NOUN_de_le_NOUN,le_NOUN_de_le,le_NOUN_de_NOUN,VERB_à_le_NOUN,un_NOUN_de_NOUN,VERB_le_NOUN_de,NOUN_à_le_NOUN,à_le_NOUN_de,le_NOUN_le_NOUN,de_le_NOUN_de
list_5_gram,le_NOUN_de_le_NOUN,le_NOUN_de_son_NOUN,VERB_le_NOUN_de_le,NOUN_de_le_NOUN_de,à_le_NOUN_de_le,VERB_à_le_NOUN_de,le_NOUN_à_le_NOUN,de_le_NOUN_de_le,le_NOUN_de_un_NOUN,NOUN_de_le_NOUN_ADJ


In [68]:
df_historique = get_motifs_by_subgenres(path_historique, chapitres_index_sentences, MOTIFS_sentences)
df_policier = get_motifs_by_subgenres(path_policier, chapitres_index_sentences, MOTIFS_sentences)
df_fantastique = get_motifs_by_subgenres(path_fantastique, chapitres_index_sentences, MOTIFS_sentences)
df_erotisme = get_motifs_by_subgenres(path_erotisme, chapitres_index_sentences, MOTIFS_sentences)
df_jeunesse = get_motifs_by_subgenres(path_jeunesse, chapitres_index_sentences, MOTIFS_sentences)
df_autobio = get_motifs_by_subgenres(path_autobio, chapitres_index_sentences, MOTIFS_sentences)
df_nonfiction = get_motifs_by_subgenres(path_nonfiction, chapitres_index_sentences, MOTIFS_sentences)
df_sf = get_motifs_by_subgenres(path_sf, chapitres_index_sentences, MOTIFS_sentences)
df_epistolaire = get_motifs_by_subgenres(path_epistolaire, chapitres_index_sentences, MOTIFS_sentences)
df_voyages = get_motifs_by_subgenres(path_voyage, chapitres_index_sentences, MOTIFS_sentences)

  0%|          | 0/84 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/249 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [70]:
df_aventures.to_csv('../metadata/df_motifs_aventures.csv')

In [71]:
df_historique.to_csv('../metadata/df_motifs_historique.csv')
df_policier.to_csv('../metadata/df_motifs_policier.csv')
df_fantastique.to_csv('../metadata/df_motifs_fantastique.csv')
df_erotisme.to_csv('../metadata/df_motifs_erotisme.csv')
df_jeunesse.to_csv('../metadata/df_motifs_jeunesse.csv')
df_autobio.to_csv('../metadata/df_motifs_autobio.csv')
df_nonfiction.to_csv('../metadata/df_motifs_nonfiction.csv')
df_sf.to_csv('../metadata/df_motifs_sf.csv')
df_epistolaire.to_csv('../metadata/df_motifs_epistolaire.csv')
df_voyages.to_csv('../metadata/df_motifs_voyages.csv')